In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50130520

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,198785.2469
2018-02-28,133635.7370
2018-03-31,120699.4307
2018-04-30,120586.9432
2018-05-31,86917.5700
2018-06-30,98578.5868
2018-07-31,99136.9987
2018-08-31,113090.6296
2018-09-30,104839.5738


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    198785.2469
 2018-02-28    133635.7370
 2018-03-31    120699.4307
 2018-04-30    120586.9432
 2018-05-31     86917.5700
 2018-06-30     98578.5868
 2018-07-31     99136.9987
 2018-08-31    113090.6296
 2018-09-30    104839.5738
 2018-10-31    108492.7573
 2018-11-30    114724.9826
 2018-12-31    216520.8939
 2019-01-31    289733.0966
 2019-02-28    153710.5255
 2019-03-31    280840.4872
 2019-04-30    148876.1619
 2019-05-31    111006.4065
 2019-06-30    124662.9986
 2019-07-31    121226.7850
 2019-08-31    152653.2825
 2019-09-30    148440.2772
 2019-10-31    155721.3967
 2019-11-30    145886.5152
 2019-12-31    157584.1871
 2020-01-31    167075.1275
 2020-02-29    165859.9964
 2020-03-31    197099.4092
 2020-04-30    135049.5860
 2020-05-31    114652.7506
 2020-06-30    117407.9316
 2020-07-31    153396.5872
 2020-08-31    136469.7123
 2020-09-30    122141.1251
 2020-10-31    165122.1058
 2020-11-30    147578.5494
 2020-12-31    188294.8838

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -3.388000
p-value : 0.011375
Critical Values :
	1%: -3.639224
	5%: -2.951230
	10%: -2.614447


In [8]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=1027.239, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=1024.575, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=1023.196, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=1019.451, Time=0.08 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.40 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=inf, Time=0.38 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=1022.351, Time=0.03 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=1024.033, Time=0.03 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=1023.326, Time=0.06 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=1022.722, Time=0.09 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=inf, Time=0.49 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=1023.616, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=1025.580, Time=0.07 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=1022.172, Time=0.13 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=inf, Time=0.49 sec
 ARIMA(3,0,0)(0,0,0)[1] interc

ARIMA(order=(0, 0, 3), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [9]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(0,0,3))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(0, 3)   Log Likelihood                -504.659
Method:                       css-mle   S.D. of innovations          39413.884
Date:                Sat, 11 Sep 2021   AIC                           1019.317
Time:                        04:37:26   BIC                           1028.006
Sample:                             0   HQIC                          1022.502
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.516e+05   1.43e+04     10.615      0.000    1.24e+05     1.8e+05
ma.L1.y        0.3975      0.141      2.827      0.005       0.122       0.673
ma.L2.y        0.5387      0.169      3.181      0.0

# 예측하기

In [10]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([137340.46499017, 151590.47980227]), array([39413.8842646 , 42413.73721755]), array([[ 60090.67134072, 214590.25863962],
       [ 68461.08240613, 234719.87719841]]))
